In [1]:
from packages.display.core import *

In [2]:
%qtconsole

# Testing Band 9

## Creation of Data Cubes

Creation of the Synthetic Data Cubes ALMA-like using ASYDO Project.
Parameters:

 - isolist    : subset of the list of isotopes to generate a cube

 cube_params:

 - freq    : spectral center (frequency)
 - alpha   : right-ascension center (degrees
 - delta   : declination center (degrees
 - spe_res : spectral resolution (MHz)
 - spe_bw  : spectral bandwidth (MHz)
 - s_f, s_a : skew-normal distrib, s_f: full width at half maximum, alpha: curtosis parameter. 

In [2]:
cube_params = {
  'freq'     : 604000,
  'alpha'    : 0,
  'delta'    : 0,
  'spe_bw'   : 4000,
  'spe_res'  : 1,
  's_f'      : 8,
  's_a'      : 0}

To select the isolist, the wavelength range of the cube is obtained, and a searching from theoretical catalog Splatalogue is performed. All the isotopes that have spectral lines on the range of the cube are determinated.

In [4]:
freq_init = cube_params['freq'] - cube_params['spe_bw']/2.0
freq_end = cube_params['freq'] + cube_params['spe_bw']/2.0

molist_present = theoretical_presence(molist, freq_init, freq_end)

Then, we get all the possible combination sets of the previously determined isotopes.

In [10]:
all_subsets = sum(map(lambda r: list(combinations(molist_present, r)),
                                range(1, len(molist_present)+1)), [])

Finally, random sets previously determinated are selected in order to generate the data cubes.

In [11]:
random_index = np.random.randint(len(all_subsets), size=25)

isolist = []
for i in random_index:
    isolist.append(all_subsets[i])
    
save_isolist(isolist)

In [3]:
isolist = load_isolist()

### Generate Datacubes in Band 9, Fixed Width

In [4]:
log=open('data/isolist_fixed_width.log', 'w')
cube_n = 0
cube_name = 'data/cube_fixed_width_'
for i in range(0, 25):

    # Creation of the cube
    gen_cube(isolist[i], cube_params, cube_name + str(cube_n))

    log.write(cube_name + ': ' + str(isolist[i]) + '\n')
    cube_n += 1
log.close()

        Use :meth:`BinTableHDU.from_columns` for new BINARY tables or :meth:`TableHDU.from_columns` for new ASCII tables instead. [packages.asydopy.vu]
        Use :meth:`BinTableHDU.from_columns` for new BINARY tables or :meth:`TableHDU.from_columns` for new ASCII tables instead.


### Generate Datacubes in Band 9, Variable (TO DO: Fix variable width in ASYDO) Width

In [9]:
log=open('data/isolist_variable_width.log', 'w')
cube_n = 25
cube_name = 'data/cube_variable_width_'
for i in range(0, 25):

    # Creation of the cube
    gen_cube_variable_width(isolist[i], cube_params, cube_name + str(cube_n))
    
    log.write(cube_name + ': ' + str(isolist[i]) + '\n')
    cube_n += 1
log.close()

## Creation of Dictionary

We create the words necessary to fit a sparse coding model to the observed spectra in the previous created cube.
It returns a DataFrame with a vector for each theoretical line for each isotope in molist

In [ ]:
dictionary = gen_words(molist, cube_params)
save_dictionary(dictionary, 'band_9')

In [3]:
dictionary = gen_words(molist, cube_params, True)
save_dictionary(dictionary, 'band_9_dual')

        Use :meth:`BinTableHDU.from_columns` for new BINARY tables or :meth:`TableHDU.from_columns` for new ASCII tables instead. [astropy.utils.decorators]
        Use :meth:`BinTableHDU.from_columns` for new BINARY tables or :meth:`TableHDU.from_columns` for new ASCII tables instead.


In [4]:
dictionary = load_dictionary('band_9')

In [5]:
dictionary = load_dictionary('band_9_dual')

## Recalibration of Dictionary

In [ ]:
for i in range(0, 50):
    if (i = 0)
        cube_name = 'data/cube_fixed_width_'
    if (i = 25)
        cube_name = 'data/cube_variable_width_cube_'
        
    file_path = cube_name + str(i) + '.fits'
    
    train_pixel = (1, 1)
    dictionary_recal = recal_words(file_path, dictionary, cube_params)
    
    X = get_values_filtered_normalized(file_path, train_pixel, cube_params)

    y_train = get_fortran_array(np.asmatrix(X))
    dictionary_recal_fa = np.asfortranarray(dictionary_recal,
                                            dtype= np.double)

### Training

In [ ]:
prediction = []

In [ ]:
for i in range(0, 50):
    param = {
      'lambda1' : 25,
      # 'L': 1,
      'pos' : True,
      'mode' : 0,
      'ols' : True,
      'numThreads' : -1}
    
    alpha = spams.lasso(y_train, dictionary_recal_fa, **param).toarray()
    total = np.inner(dictionary_recal_fa, alpha.T)
    save_prediction(alpha, cube_name + str(i))

In [ ]:
for i in range(0, 50):
    prediction[i] = load_prediction(cube_name + str(i))

### Testing

In [ ]:
confusion_matrix = []
results = []

In [ ]:
for i in range(0, 50):
    confusion_matrix[i], results[i] = test(dictionary_recal, alpha, file_path, cube_params)

In [14]:
confusion_matrix[0]

,SO2v2=1-f605781.546,SO2v2=1-f603981.474,H213CO-f602083.584,SO2v2=1-f604315.0048,SO2v2=1-f605984.3296,SO2v2=1-f602454.6493,SO2v2=1-f604398.1207,H2C34S-f605585.4505,H2C34S-f604710.9998,H13CNv=0-f604267.914,H2C34S-f605200.1552,H2C18O-f603231.51,SO2v2=1-f603332.8487,SO2v2=1-f604022.2577,H2C34S-f605935.8198,SO2v2=1-f605850.4063,H213CS-f602149.9892,SO2v2=1-f604728.5953
CH3OHvt=0-f602233.197,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
34SO2v=0-f604376.563,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
H213CO-f602083.584,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
results[0]

,Precision,Recall,F-Score
SO2v2=1-f605781.546,0,0,0
SO2v2=1-f603981.474,0,0,0
H213CO-f602083.584,0,0,0
SO2v2=1-f604315.0048,0,0,0
SO2v2=1-f605984.3296,0,0,0
SO2v2=1-f602454.6493,0,0,0
SO2v2=1-f604398.1207,0,0,0
H2C34S-f605585.4505,0,0,0
H2C34S-f604710.9998,0,0,0
H13CNv=0-f604267.914,0,0,0


# Testing Band 7

### Generate Datacubes in Band 7, Fixed Width

In [12]:
cube_params['freq'] = 277000

In [13]:
log=open('data/isolist_fixed_width.log', 'w')
cube_n = 50
cube_name = 'data/cube_fixed_width_'
for i in range(0, 25):

    # Creation of the cube
    gen_cube(isolist[i], cube_params, cube_name + str(cube_n))

    log.write(cube_name + ': ' + str(isolist[i]) + '\n')
    cube_n += 1
log.close()

### Generate Datacubes in Band 7, Variable (TO DO: Fix variable width in ASYDO) Width

In [16]:
log=open('data/isolist_fixed_width.log', 'w')
cube_n = 75
cube_name = 'data/cube_variable_width_'
for i in range(0, 25):

    # Creation of the cube
    gen_cube_variable_width(isolist[i], cube_params, cube_name + str(cube_n))

    log.write(cube_name + ': ' + str(isolist[i]) + '\n')
    cube_n += 1
log.close()

## Creation of Dictionary

In [14]:
dictionary = gen_words(molist, cube_params)
save_dictionary(dictionary, 'band_7')

In [ ]:
dictionary = load_dictionary('band_7')

### Recalibration of the Dictionary

In [ ]:
for i in range(0, 50):
    # #
    file_path = cube_name + str(cube_n) + '.fits'
    
    train_pixel = (1, 1)
    dictionary_recal = recal_words(file_path, dictionary, cube_params)
    
    X = get_values_filtered_normalized(file_path, train_pixel, cube_params)

    y_train = get_fortran_array(np.asmatrix(X))
    dictionary_recal_fa = np.asfortranarray(dictionary_recal,
                                            dtype= np.double)

### Training

In [ ]:
for i in range(50, 100):
    param = {
      'lambda1' : 25,
      # 'L': 1,
      'pos' : True,
      'mode' : 0,
      'ols' : True,
      'numThreads' : -1}
    
    alpha = spams.lasso(y_train, dictionary_recal_fa, **param).toarray()
    total = np.inner(dictionary_recal_fa, alpha.T)
    save_prediction(alpha, cube_name + str(cube_n))

In [ ]:
for i in range(50, 100):
    prediction[i] = save_prediction(alpha, cube_name + str(cube_n))

### Testing

In [ ]:
results = []
for i in range(50, 100):
    confusion_matrix[i], results[i] = test(dictionary_recal, alpha, file_path, cube_params)

In [ ]:
confusion_matrix[50]

In [ ]:
results[50]

In [2]:
from astropy.io import fits